In [96]:
import pandas as pd
import altair as alt
import numpy as np

import sqlite3 as sql

import json

# Use to handle large amount of data
# alt.data_transformers.enable('json')

In [97]:
# Machine learning model
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

# Data sets used
These are the data sets used in the outline for the examples code.

In [98]:
cars_url = "https://github.com/byuidatascience/data4missing/raw/master/data-raw/mtcars_missing/mtcars_missing.json" # this gets the cars data from this url
car_db = pd.read_json(cars_url) # uses pandas read_json command to open the data set in pandas format

names_url = "names_year.csv"
names_db = pd.read_csv(names_url)
alt.data_transformers.enable('json')

con = sql.connect("lahmansbaseballdb.sqlite") # allows you to us the sql data set

# Line chat 
how to get a line chat to show up in alt air

In [99]:
all_names_wa= names_db.filter(["name", "year", "WA"])
name_list = ["Aarna", "Aaden", "Sheldon", "Nathan", "Ben"]
all_names_wa = names_db.query("name == @name_list")
all_names_wa_chart = (alt.Chart(all_names_wa, title = "names from Washington line chart")
                      .mark_line()
                      .encode(
                          x = alt.X("year:O", title= "year of use"), # use the :O to make the numbers not have a , in them
                          y = alt.Y("WA", title= "Num times name was used"),
                          color = "name"
                      )
                      )
all_names_wa_chart

alt.Chart(...)

# line chart templet
all_names_wa_chart = (alt.Chart(, title = "")
                      .mark_line()
                      .encode(
                          x = alt.X("", title= ""), 
                          y = alt.Y("WA", title= ""),
                          color = "name")
                          )

# bar chart 

In [100]:
bar_names = names_db.copy()
bar_names = bar_names.filter(["name", "year", "WA"])
name_list = ["Aarna", "Aaden", "Sheldon", "Nathan", "Ben"]
bar_names = bar_names.query("name == @name_list")

bar_names = bar_names.groupby("name").agg(
    avg_num_name_use_per_year = ("WA", np.mean),
    sum_name_use = ("WA", np.sum)
).reset_index().round(0)

bar_names_chart = (alt.Chart(bar_names, title  = "names in washington bar chart")
                   .mark_bar()
                   .encode(
                       x = alt.X("name", title = "names"),
                       y = alt.Y("sum_name_use", title= "sum_use")
                   ))
bar_names_chart


alt.Chart(...)

# bar chart templet
 = (alt.Chart(, title  = "")
                   .mark_bar()
                   .encode(
                       x = alt.X("", title = ""),
                       y = alt.Y("", title= "")
                   ))

# adding avg line and adding text
We will use the chart from bar chart for this example

In [101]:
bar_names = names_db.copy()
bar_names = bar_names.filter(["name", "year", "WA"])
name_list = ["Aarna", "Aaden", "Sheldon", "Nathan", "Ben"]
bar_names = bar_names.query("name == @name_list")
names_avg = (bar_names.WA.sum() / len(name_list)).round(0)
bar_names = bar_names.assign(
    avg_name_use_over_all = names_avg
)
bar_names

avg_line = (alt.Chart(bar_names)
            .mark_line(color= "RED")
            .encode(
                x= alt.X("name"),
                y = alt.Y("avg_name_use_over_all")
            ))

bar_text = bar_names_chart.mark_text(
    angle = 90,
    dx = -15
    ).encode(
    text = alt.Text("sum_name_use:Q")
)


bar_names_chart + avg_line + bar_text  # add the charts in the right order or it will not work right


alt.LayerChart(...)

# show %
on the axis you want to show in % use format= (",.0%") in side of alt.X OR alt.Y or you could use axis=alt.Axis(format='%') also inside alt.X or alt.Y

# coding challenge


alt.Chart(...)

# SQL

### Explaing SQL

'SELECT' - Saying what data you want to pull

'*' - Pulls all tables

'FROM' - Which table you want to pull from

'fielding' - Table you are pulling from

'LIMIT 5' - only show '5' rows

con - sql database

In [78]:
df = pd.read_sql_query("""SELECT * 
                        FROM fielding 
                        LIMIT 5""", con)
df

,ID,playerID,yearID,stint,teamID,team_ID,lgID,POS,G,GS,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR
0,1,abercda01,1871,1,TRO,8,NA,SS,1,1,24,1,3,2,0,None,None,None,None,None
1,2,addybo01,1871,1,RC1,7,NA,2B,22,22,606,67,72,42,5,None,None,None,None,None
2,3,addybo01,1871,1,RC1,7,NA,SS,3,3,96,8,14,7,0,None,None,None,None,None
3,4,allisar01,1871,1,CL1,3,NA,2B,2,0,18,1,4,0,0,None,None,None,None,None
4,5,allisar01,1871,1,CL1,3,NA,OF,29,29,729,51,3,7,1,None,None,None,None,None


### Creating New Dataframe

When creating a new dataframe query the data you need and set it to a new variable

The LIMIT will effect the dataframe

In [79]:
sql_dataframe_0 = pd.read_sql_query("""SELECT teamID
                                FROM Teams
                                LIMIT 5
                                """,
                                con)

sql_dataframe_0

,teamID
0,BS1
1,CH1
2,CL1
3,FW1
4,NY2


### Finding Avgerage

'ROUND' - A round function

'cast' - Converts a value to a new datatype

'ORDER BY' - Use to sort the results

'DESC' - Use to order by a descending order

In [80]:
sql_dataframe_2 = pd.read_sql_query("""SELECT playerID, yearID, ROUND(sum((cast(H as real) / ab)), 3) AS "Batting Average" 
                            FROM Batting
                            WHERE ab >= 100
                            GROUP BY playerID
                            ORDER BY "Batting Average" DESC, playerID
                            LIMIT 5
                            """,
                            con)

sql_dataframe_2

,playerID,yearID,Batting Average
0,ansonca01,1871,9.131
1,cobbty01,1905,8.663
2,henderi01,1979,7.376
3,rosepe01,1963,7.152
4,speaktr01,1908,7.104


### SQL to Altair

'SUM' - is a sum function for sql

'AS' - Renaming the column

'WHERE' - Specifying data you want to use

'OR' - standard or function

'GROUP BY' - filtering by a column

In [81]:
sql_dataframe_1 = pd.read_sql_query("""SELECT teamID, SUM(W) AS "Total Games Won"
                            FROM Teams
                            WHERE teamID = "ALT" OR teamID = "BFN"
                            GROUP BY teamID
                            LIMIT 10
                            """,
                            con)

sql_dataframe_1

,teamID,Total Games Won
0,ALT,6
1,BFN,314


#### Chart Creation

Use altair to creat a chart that was created by SQl

In [82]:
sql_chart_0 = (alt.Chart(sql_dataframe_1)
    .encode(
        x= alt.X('teamID'),
        y= alt.Y('Total Games Won'),
        color= 'teamID'
    ).mark_bar())

# Saving the chart as a png
sql_chart_0.save("screenshots/sql_chart_0.png")

sql_chart_0

ValueError: No enabled saver found that supports format='png'

# Markdown Notes

#### Printing tables to MD

Use the to_markdown() functions

In [ ]:
print(sql_dataframe_0.to_markdown())

|    | teamID   |
|---:|:---------|
|  0 | BS1      |
|  1 | CH1      |
|  2 | CL1      |
|  3 | FW1      |
|  4 | NY2      |
